In [1]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing import image



2024-09-28 22:11:51.124397: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('data/styles.csv', error_bad_lines=False)
image_directory = 'data/images'


/var/folders/16/d8sd9tw11_q8c7jxbcz_m6480000gn/T/ipykernel_68627/1580367284.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('data/styles.csv', error_bad_lines=False)
Skipping line 6044: expected 10 fields, saw 11
Skipping line 6569: expected 10 fields, saw 11
Skipping line 7399: expected 10 fields, saw 11
Skipping line 7939: expected 10 fields, saw 11
Skipping line 9026: expected 10 fields, saw 11
Skipping line 10264: expected 10 fields, saw 11
Skipping line 10427: expected 10 fields, saw 11
Skipping line 10905: expected 10 fields, saw 11
Skipping line 11373: expected 10 fields, saw 11
Skipping line 11945: expected 10 fields, saw 11
Skipping line 14112: expected 10 fields, saw 11
Skipping line 14532: expected 10 fields, saw 11
Skipping line 15076: expected 10 fields, saw 12
Skipping line 29906: expected 10 fields, saw 11
Skipping line 31625: expected 10 fields, saw 11
Ski

In [3]:
df.head()


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [6]:
df = df[~df['usage'].isna()]

df['id'] = df['id'].astype(str)
df['id'] = df['id'] + '.jpg'

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


In [7]:
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=image_directory,
    x_col='id',
    y_col='usage',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=image_directory,
    x_col='id',
    y_col='usage',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 35282 validated image filenames belonging to 8 classes.


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 5 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  warnings.warn(


Found 8820 validated image filenames belonging to 8 classes.


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 5 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  warnings.warn(


In [8]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n//train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.n//validation_generator.batch_size,
    epochs=10
)


Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1102/1102 ━━━━━━━━━━━━━━━━━━━━ 284s 255ms/step - accuracy: 0.8088 - loss: 0.7050 - val_accuracy: 0.8750 - val_loss: 0.3924
Epoch 2/10
1102/1102 ━━━━━━━━━━━━━━━━━━━━ 0s 114us/step - accuracy: 0.8750 - loss: 0.4514 - val_accuracy: 0.9500 - val_loss: 0.4843
Epoch 3/10


2024-09-28 22:17:06.516313: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-09-28 22:17:06.611732: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


1102/1102 ━━━━━━━━━━━━━━━━━━━━ 274s 249ms/step - accuracy: 0.8677 - loss: 0.4125 - val_accuracy: 0.8834 - val_loss: 0.3527
Epoch 4/10
1102/1102 ━━━━━━━━━━━━━━━━━━━━ 0s 63us/step - accuracy: 0.9375 - loss: 0.2405 - val_accuracy: 0.9500 - val_loss: 0.2234
Epoch 5/10


...

2024-09-28 22:26:21.822912: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-09-28 22:26:21.880431: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


1102/1102 ━━━━━━━━━━━━━━━━━━━━ 296s 269ms/step - accuracy: 0.8920 - loss: 0.3140 - val_accuracy: 0.8916 - val_loss: 0.3301
Epoch 8/10
1102/1102 ━━━━━━━━━━━━━━━━━━━━ 0s 68us/step - accuracy: 0.9375 - loss: 0.5107 - val_accuracy: 0.8500 - val_loss: 0.3359
Epoch 9/10


2024-09-28 22:31:18.392335: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-09-28 22:31:18.449841: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


1102/1102 ━━━━━━━━━━━━━━━━━━━━ 285s 259ms/step - accuracy: 0.9033 - loss: 0.2736 - val_accuracy: 0.8947 - val_loss: 0.3270
Epoch 10/10
1102/1102 ━━━━━━━━━━━━━━━━━━━━ 0s 66us/step - accuracy: 0.9375 - loss: 0.1716 - val_accuracy: 0.8000 - val_loss: 0.5077


2024-09-28 22:36:04.091802: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-09-28 22:36:04.148491: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation accuracy: {accuracy * 100}%")


In [ ]:
img = image.load_img('data/images/1757.jpg', target_size=(128, 128))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

predictions = model.predict(img_array)
predicted_class = np.argmax(predictions[0])

for style, index in train_generator.class_indices.items():
    if index == predicted_class:
        print(f"Predicted Style: {style}")
